# Assignement 3 Report

#### Anja Deric and Karan Soni

### ABSTRACT

For this assignment, we worked on collecting data from social media to complement our already existing data on flights, airports and airlines. Additionally, we also converted our previous database to NoSQL by connecting to a MongoDB server. FOr the first part of the assignment, we used the Twitter API to collect data on various topics related to our domain, including tweets about American Airlines (thing), Logan Airport (place), and pilots (people). We used Python to analyze this tweet data and determine what tags were most popular and trending within our domain. For the second half of the assignment, we imported our old data from Assignments 1 and 2, and along with our newly-collected twitter data, we converted everything into a NoSQL database. By connecting to a MongoDB server, we were also able to test and make sure some of database usecases worked and could provide information requested by the user.


### Collecting Social Media Data

To start the assignment, we first imported all libraries that we would be using throughout:

In [239]:
import csv
import json
import pandas as pd
import tweepy
import re
import numpy as np
import pandas as pd
import collections

We then made a twitter developer account and used our keys and tokens to establish a connection with the Twitter API.

In [4]:
# API keys and tokens
consumer_key = "mGtIt09UVXyzyW5LMBx6YKSIg"  
consumer_secret = "DCi2axya3I6iRdxLvnNzPbInscCA7oquTaZKJqSK2WOCDriwjp"  
access_token = "1238579197867524096-EhwxltGCsYmzWsDYoW9JsIqT3Yghck"  
access_token_secret = "2EBkSz2GRrC8w0tdpPxKEVGC4swpAQsDDoNkUMbKresIg"

# Establish connection with twitter API using developer keys
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)  
auth.set_access_token(access_token, access_token_secret)  
api = tweepy.API(auth, wait_on_rate_limit=True)

Next, we wrote a function to collect 100 tweets (combination of most popular and most recent tweets) by making an API request using the Cursor function from the tweepy library. The function works by searching for tweets that have a specific search term in them, which the user can decide on. Each tweet is then stored in a dictionary (we collected information such as the username, tweet test, number of retweets, hashtags, etc.). The function, in the end, returns all the tweets as a dataframe.

In [175]:
# Funtion to collect information on 100 tweets that contain a particular search term
def get_tweets(search_term):
    all_tweets = []
    # Make API request for tweets in English that contain search_term
    for tweet in tweepy.Cursor(api.search, q=search_term,lang = "en").items(100):
        # Store all tweets in dictionary
        all_tweets += [{ 'Tweet_id': tweet.id,
                'Screen_name':tweet.author.screen_name,
                'Created_at':tweet.created_at,
                'Tweet_text':tweet.text,
                'Hashtags':re.findall(r"#(\w+)",tweet.text),
                'Retweets':tweet.retweet_count,
                'Favorites':tweet.favorite_count,
                'Location':tweet.user.location}]
    # Return all tweets in dataframe format
    return pd.DataFrame(all_tweets)

Then, we used our function to collect 100 tweets relating to American Airlines, Logan Airport, and pilots, all terms related to our domain. To ensure our funciton worked, we additionally used the .head() method to check the first few tweets.

In [176]:
# Get 100 tweews on American Airlines (thing)
all_tweets = get_tweets("#AmericanAirlines")
airline_tweets = all_tweets
airline_tweets.head()

,Tweet_id,Screen_name,Created_at,Tweet_text,Hashtags,Retweets,Favorites,Location
0,1247229042371297281,AMERICA_PARTII,2020-04-06 18:25:50,RT @KelliRenard: American Arlines won’t refund...,[],10,0,
1,1247227986627215362,michael83747287,2020-04-06 18:21:39,Thanks @Delta @SouthwestAir and @AlaskaAir for...,[],0,0,
2,1247225304864743424,Cjkbeauty,2020-04-06 18:10:59,Fighting with @AmericanAir all day to get a re...,[AmericanAirlines],0,0,"Pittsburgh, Pa"
3,1247222584468557826,Lake25246548,2020-04-06 18:00:11,https://t.co/d7XCgMIYkl\nFarewell 757/adios 75...,[],0,0,
4,1247215366088134656,PKOPEC17,2020-04-06 17:31:30,RT @AJC4others: We may have found out first bo...,[AmericanAirlines],2,0,"None, Piemonte"


In [177]:
# Get 100 tweews on Logan Airport (place)
all_tweets = get_tweets("#LoganAirport")
airport_tweets = all_tweets
airport_tweets.head()

,Tweet_id,Screen_name,Created_at,Tweet_text,Hashtags,Retweets,Favorites,Location
0,1246797293338136576,RWayneLopez,2020-04-05 13:50:13,"Coming Soon 45 Province PH3B for lease at $12,...",[],0,1,"Boston, MA"
1,1246614553535922181,BOS_Boston_Limo,2020-04-05 01:44:05,"First class premium car services Boston, MA\nh...",[loganairport],0,1,"Boston, MA"
2,1246595470417371138,LisaAFerrari,2020-04-05 00:28:15,Looking for the place that moved so I can drop...,"[loganairport, bostonfishpier]",0,1,"Boston,MA"
3,1246164233240354816,jb_SID,2020-04-03 19:54:40,"26,000 people departed #LoganAirport last week...","[LoganAirport, Covid]",0,0,Boston
4,1246117138659545089,getmybuzzup,2020-04-03 16:47:32,The #newenglandpatriots plane just touched dow...,"[newenglandpatriots, loganairport, boston, n95...",0,0,"Yonkers, NY, CT, NJ, ATL, LA"


In [98]:
# Get 100 tweews on Pilot (person)
all_tweets = get_tweets("#pilot")
pilot_tweets = all_tweets
pilot_tweets.head()

,Tweet_id,Screen_name,Created_at,Tweet_text,Hashtags,Retweets,Favorites,Location
0,1247210121354186759,saltydogsbot,2020-04-06 17:10:39,RT @JadeEyePanda: #Titanfall #fanart Legionnai...,"[Titanfall, fanart, Modo, 3DArt, art, mecha, m...",1,0,
1,1247209812389224452,JadeEyePanda,2020-04-06 17:09:25,#Titanfall #fanart Legionnaire WIP 4 #Modo #3D...,"[Titanfall, fanart, Modo, 3DArt, art, mecha, m...",1,0,"Buena Park, CA"
2,1247208734767616003,luisevanegas,2020-04-06 17:05:09,RT @Fiicaan: Kalitta operated month old 777F h...,"[avgeek, aviationphoto]",10,0,Desearía estar en #MiEspaña
3,1247207541173555200,aeromarinetax,2020-04-06 17:00:24,"Aircraft, Vessels, or Vehicles-How Out of Stat...",[],0,0,"Elk Grove, CA"
4,1247204199647055872,PlaneSpotIsCool,2020-04-06 16:47:07,RT @BBSAirport: We can’t wait to get back to d...,"[StaySafe, Airport, Timelapse, BusinessAviatio...",1,0,Worldwide


### Social Media Questions

After collecting all the data, we analyzed it in order to answer questions about tags, users, and trending topic within our domain. Below are the answers to all the questions from the assignment as well as detailed descriptions on how we went about finding those answers.

#### What are tags are associated with a person, place or thing?

To determine what tags were associated with each of our data frames, we first a function to extract tags from our data frame. The get_hashtags function loops through every row of the dataframe and looks at the 'Hashtag' column. Since some tweets have multiple tags, the function splits them up and replaces or removes all the blank spots. Each tag is then added to the overall list of tags and sent back.

Now that this functions was created, we could simply call it with each domain- airport, airline, pilot.

In [114]:
# Function to extract all tags from the data frame
def get_hashtags(tweets):
    all_hashtags = []
    # Loop through each row in data frame
    for index, row in tweets.iterrows():
        # Extract all tags and split them up
        tweet_hashtags = row['Hashtags']
        for tag in tweet_hashtags:
            # Remove any extra space and add to the list of all tags
            tag = tag.replace("'", "")
            all_hashtags.append(tag.replace(" ", ""))
 
    # Remove all blank entries
    while("" in all_hashtags) : 
        all_hashtags.remove("")
        
    return all_hashtags

Here all the tags associated with American Airlines (a thing):

In [133]:
# Find and print most popular tags for American Airlines 
airline_tags = get_hashtags(airline_tweets)
print(airline_tags)

['americanairlines', 'covid', 'AmericanAirlines', 'Stock', 'AmericanAirlines', 'AmericanAirlines', 'AmericanAirlines', 'AmericanAirlines', 'AmericanAirlines', 'AmericanAirlines', 'COVID2019', 'Ame', 'AmericanAirlines', 'refund', 'insurance', 'Paramedic', 'americanairlines', 'AmericanAirlines', 'americanairlines', 'AmericanAirlines', 'AmericanAirlines', 'Stock', 'MarketScre', 'AmericanAirlines', 'AmericanAirlines', 'coronavirus', 'NewYork', 'AmericanAirlines', 'American', 'AmericanAirlines', 'FrequentFlierBenefits', 'FFbenefits', 'AmericanAirlines', 'AAfamily', 'AAfrequentflier', 'AmericanAirlines', 'coronavirus', 'NewYork', 'AmericanAirlines', 'United', 'NewYork', 'Flights', 'AviationNews', 'AmericanAirlines', 'virus', 'AmericanAirlines', 'Cargo', 'AmericanAirlines', 'COVID19', 'AmericanAirlines', 'AmericanAirlines', 'BusinessTravel', 'A', 'lax', 'klax', 'americanairlines', 'airplane', 'Boeing', 'Boeing777', 'AmericanAirlines', 'toky', 'RJTT', 'Narita', 'AmericanAirlines', 'Qantas', 'A

Here all the tags associated with Logan Airport (a place):

In [134]:
# Find and print most popular tags for Logan Airport
airport_tags = get_hashtags(airport_tweets)
print(airport_tags)

['loganairport', 'loganairport', 'bostonfishpier', 'LoganAirport', 'Covid', 'newenglandpatriots', 'loganairport', 'boston', 'n95mask', 'repost', 'NewEngland', 'Patriots', 'LoganAirport', 'Boston', 'repost', 'NewEngland', 'Patriots', 'LoganAirport', 'Boston', 'repost', 'NewEngland', 'Patriots', 'LoganAirport', 'Boston', 'N95masks', 'Boston', 'LoganAirport', 'loganairport', 'loganairport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'LoganAirport', 'quincycab', '6174718294', 'quincyma', 'quincycab', '6174718294', 'quincycab', '6174718294', 'quincycab', '6174718294', 'loganairport', 'FlyingHome']


Here all the tags associated with pilots (a person):

In [137]:
# Find and print most popular tags for pilot
pilot_tags = get_hashtags(pilot_tweets)
print(pilot_tags)

['Titanfall', 'fanart', 'Modo', '3DArt', 'art', 'mecha', 'mech', 'respawn', 'gameart', 'gamedev', 'robot', 'military', 'scifi', 'l', 'Titanfall', 'fanart', 'Modo', '3DArt', 'art', 'mecha', 'mech', 'respawn', 'gameart', 'gamedev', 'robot', 'military', 'avgeek', 'aviationphoto', 'StaySafe', 'Airport', 'Timelapse', 'BusinessAviation', 'GeneralAv', 'StaySafe', 'Airport', 'Timelapse', 'BusinessAviation', 'FlightDeckMonday', 'flight', 'Pilot', 'IFR', 'businesscharter', 'privatejet', 'luxurymarbella', 'Comedy', 'Dramedy', 'TV', 'Series', 'pilot', 'episode', 'bible', 'Hollywood', 'studio', 'Comedy', 'Dramedy', 'TV', 'Series', 'pilot', 'episode', 'bible', 'Hollywood', 'pilot', 'publications', 'pilotlife', 'pilot', 'piloteyes', 'aviation', 'pilotview', 'instapilot', 'lifeasap', 'avgeek', 'aviationphotography', 'planespott', 'pilotlife', 'pilot', 'piloteyes', 'aviation', 'pilotview', 'instapilot', 'lifeasap', 'pilotlife', 'pilot', 'piloteyes', 'aviation', 'pilotview', 'video', 'pilot', 'mountaino

#### What social media users are like other social media users in your domain?

To determine which users in our domain are similar, we created a function called get_similar_users. The function takes in the data frame with all the tweet information and finds all users that tweeted using a particular tag. This is done by looping through all the tags, and any time the desired tag is found, the user that made that tweet is added to the list of similar users. This way, all users that tweet about a particular topic and use its specific tag are deemed to be similar.

In [124]:
# Function to find all users that used the same tag in their tweets
def get_similar_users(tweets, desired_tag):
    similar_users = []
    # Loop through each row in data frame
    for index, row in tweets.iterrows():
        # Extract all tags and split them up
        tweet_hashtags = row['Hashtags']
        for tag in tweet_hashtags:
            # Remove any extra space from tag
            tag = tag.replace("'", "")
            tag = tag.replace(" ", "")
            # If the tag matches the desired tag, add user to the list of similar users
            if tag == desired_tag:
                similar_users.append(row['Screen_name'])
        
    return similar_users

Here are some similar users which all tweeted about B757- a specific airplane in American Airlines' fleet:

In [131]:
# Find all distinct users that tweeted about B757
airline_users = get_similar_users(airline_tweets,'B757')
for user in set(airline_users):
    print(user)

JordanWeir45
PlaneSpotIsCool
Peter34080554
TyphoonDobby
avgeekjoseph
NoCoffeeHere
avgeek__
iuY0jx47oBZfIlO


Here are some similar users which all tweeted about Quincy Cab- a specific cab company at Boston's Logan Airport:

In [132]:
# Find all distinct users that tweeted about Quincy Cab
airport_users = get_similar_users(airport_tweets,'quincycab')
for user in set(airport_users):
    print(user)

365SouthShore
YellowTaxiNow


Here are some similar users which all tweeted about aviation photography:

In [130]:
# Find all distinct users that tweeted aviation photos
pilot_users = get_similar_users(pilot_tweets,'aviationphoto')
for user in set(pilot_users):
    print(user)

jetcitystar
luisevanegas
cargoeu
aldersonjackson
LandingsNL


#### What people, places or things are popular in your domain?

To get a list of most popular items in each domain, we created a function called popular_tags. This function uses the Counter method from the collections library to get a list of each tag used, accompanied by how many times it was used. It then uses the most_common method to extract the top 10 tags and print the tags (as well as the number of times they were used). This enabled us to see what topics were most popular for each set of tweets.

To get most popular people in the domain, we created a function called popular_user. This function gets all usernames and their associated popularity (calculated by adding up the number of retweets and favorites). Then, it finds the top 3 users and prints their name as well as popularity (sum of retweets and favorites for a single tweet).

Using these 2 functions, we were able to determine the most popular people, places, and things.

In [222]:
# Function to get most popular tags
def popular_tags(all_tags):
    # Count all tags and get top 10 most used tags
    tag_counts = collections.Counter(all_tags)
    popular_tags = tag_counts.most_common(10)
    # Print tags to the console
    for tag in popular_tags:
        print(tag[0] + ", " + str(tag[1]))
   
# Function to find top users (num specifies how many top users to find)
def popular_users(tweets, num):
    users = []
    popularity = []
    # Loop through each row in data frame
    for index, row in tweets.iterrows():
        # Calculate popularity as sum of retweets and favorites
        users.append(row['Screen_name'])
        popularity.append(row['Retweets'] + row['Favorites'])
    
    # Find maximum popularity values and the associated users 
    for index in range(len(popularity)):
        top_index = np.argsort(popularity)[-num:]
        top_users = [users[i] for i in top_index]
        top_popularity = [popularity[i] for i in top_index]
    
    # Print top users and their popularity
    for index in range(len(top_users)):
        print("User: "+top_users[index]+", Popularity: "+str(top_popularity[index]))

Here are the most popular people, places, and things in the American Airline domain:

In [223]:
# Find popular tags
print("Populart Places and Things")
popular_tags(airline_tags)

# Find top 5 popular users
print()
print("Popular Users")
popular_users(airline_tweets,5)

Populart Places and Things
AmericanAirlines, 33
B757, 8
americanairlines, 4
refund, 4
insurance, 4
Paramedic, 4
Stock, 3
NewYork, 3
COVID19, 3
Boeing, 3

Popular Users
User: lantech19, Popularity: 10
User: AMERICA_PARTII, Popularity: 10
User: victorio_bdx, Popularity: 14
User: sammy_palmerrr, Popularity: 929
User: tothanines, Popularity: 929


Here are the most popular people, places, and things in the Logan Airport domain:

In [225]:
# Find popular tags
print("Populart Places and Things")
popular_tags(airport_tags)

# Find top 5 popular users
print()
print("Popular Users")
popular_users(airport_tweets,5)

Populart Places and Things
LoganAirport, 25
loganairport, 6
Boston, 4
quincycab, 4
6174718294, 4
repost, 3
NewEngland, 3
Patriots, 3
bostonfishpier, 1
Covid, 1

Popular Users
User: Winthropvikings, Popularity: 20
User: chipsy231, Popularity: 20
User: doogs1227, Popularity: 20
User: winthropma02152, Popularity: 20
User: JacqueGoddard, Popularity: 31


Here are the most popular people, places, and things in the pilot domain:

In [226]:
# Find popular tags
print("Populart Places and Things")
popular_tags(pilot_tags)

# Find top 5 popular users
print()
print("Popular Users")
popular_users(pilot_tweets,5)

Populart Places and Things
pilot, 26
aviation, 20
avgeek, 13
airplane, 9
planes, 9
Pilot, 7
jets, 7
flying, 6
aviationphoto, 5
aviationphotography, 5

Popular Users
User: flyAPM, Popularity: 22
User: wolfmaster328, Popularity: 22
User: aldersonjackson, Popularity: 32
User: EritreanJournal, Popularity: 43
User: aldersonjackson, Popularity: 55


#### What people, places or things are trending in your domain?

In [236]:
def get_tweets_until(search_term,date):
    all_tweets = []
    for tweet in tweepy.Cursor(api.search, q=search_term,lang = "en",until=date).items(100):
        #tweet_id = tweet.id
        all_tweets += [{ 'Tweet_id': tweet.id,
                'Screen_name':tweet.author.screen_name,
                'Created_at':tweet.created_at,
                'Tweet_text':tweet.text,
                'Hashtags':re.findall(r"#(\w+)",tweet.text),
                'Retweets':tweet.retweet_count,
                'Favorites':tweet.favorite_count,
                'Location':tweet.user.location}]
    return pd.DataFrame(all_tweets)

In [237]:
tag_trend = {1:[],2:[],3:[],4:[],5:[],6:[],7:[]}
for day in range(7):
    date = "2020-04-0" + str(day+1)
    tweets = get_tweets_until("#airport",date)
    all_tags = get_hashtags(tweets)
    tag_counts = collections.Counter(all_tags)
    popular_tags = tag_counts.most_common(10)
    tag_trend.update({day+1: popular_tags})
    

In [238]:
test =json.dumps(tag_trend)
print(test)

{"1": [["airport", 48], ["Orly", 22], ["Airport", 14], ["construction", 12], ["infrastructure", 12], ["Paris", 4], ["avgeek", 3], ["COVID19", 3], ["coronavirus", 3], ["covid", 3]], "2": [["airport", 26], ["Airport", 13], ["aviation", 11], ["Airline", 9], ["Aviation", 8], ["avgeek", 6], ["COVID", 6], ["plane", 5], ["EmiratesSkyCargo", 5], ["cargo", 5]], "3": [["news", 18], ["Airport", 16], ["airport", 11], ["Business", 11], ["travel", 8], ["SB19_JOSH", 8], ["aviation", 8], ["avgeek", 8], ["airplane", 6], ["vacation", 5]], "4": [["Airport", 22], ["airport", 18], ["\ud150", 18], ["\u674e\u6c38\u94a6", 18], ["TEN", 18], ["\u0e40\u0e15\u0e19\u0e25", 18], ["NCT127", 13], ["\uc5d4\uc528\ud2f012", 13], ["swfl", 10], ["aviation", 10]], "5": [["airport", 21], ["Airport", 13], ["Boeing", 11], ["\ud150", 8], ["\u674e\u6c38\u94a6", 8], ["TEN", 8], ["\u0e40\u0e15\u0e19\u0e25", 8], ["KLM", 8], ["RoyalDutchAirlines", 8], ["Boeing787", 8]], "6": [["Airport", 27], ["airport", 23], ["Florida", 20], ["For

### Transitioning to NoSQL

In [261]:
import pymongo
import pprint

In [245]:
client = pymongo.MongoClient('localhost', 27017)
db = client['assignment3']
collection=db['tweets']
tweets = db.tweets

In [257]:
tweet_combo = [airline_tweets,airport_tweets,pilot_tweets]
all_tweets = pd.concat(tweet_combo)

In [258]:
tweets.insert_many(all_tweets.to_dict('records'))

In [259]:
db.list_collection_names(include_system_collections=False)

['tweets']

In [262]:
for tt in tweets.aggregate([ {"$group": {"_id": "$Hashtags", "count": {"$sum": 1}}},{"$sort":{"count":-1}}  ]):
    pprint.pprint(tt)

{'_id': [], 'count': 111}
{'_id': ['LoganAirport'], 'count': 20}
{'_id': ['AmericanAirlines'], 'count': 20}
{'_id': ['avgeek', 'aviationphoto'], 'count': 10}
{'_id': ['avgeek', 'aviationphotography', 'planespott'], 'count': 8}
{'_id': ['refund', 'insurance', 'Paramedic'], 'count': 4}
{'_id': ['wingtipchallenge',
         'aviation',
         'flying',
         'fun',
         'pilot',
         'sling2aircraft',
         'pilottraining'],
 'count': 4}
{'_id': ['aviation', 'airplane', 'planes'], 'count': 4}
{'_id': ['bafta',
         'premiere',
         'pilot',
         'screening',
         'interneurs',
         'tvseries',
         'e'],
 'count': 4}
{'_id': ['B757'], 'count': 4}
{'_id': ['pilot', 'flying'], 'count': 4}
{'_id': ['MaltaMaritimePilots',
         'Safehaven',
         'Marine',
         'built',
         'Pilot',
         'boat',
         'FOXTROT',
         'Grand',
         'Harbour',
         'Malta'],
 'count': 4}
{'_id': ['pilotlife',
         'pilot',
         'p

### REPORT

Blah, balh


### CONCLUSION

The primary focus of this assignment was learnign how to analyze social media data, as well as convert a SQL/relational database into a NoSQL database (MongoDB, specifically). We expanded upon our database from Assignment 2 by adding twitter data to it, and analyzing it in order to answer questions about popular topic within our domain. We then converted our database into MongoDB by first denormalizing it, and then establishing a connection with a MongoDB server and transfering over our data.

### CONTRIBUTION
On our own: 

External Sources: 

Provided by Professor:  

### CITATIONS

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets (Twitter API Documentation- help with collecting Twitter data)



### LICENSE
Copyright 2020 Anja Deric, Karan Soni

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software without restriction, including without limitation the 
rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit
persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the 
Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE 
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR 
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.